In [2]:
import datetime
import pandas as pd
import globus_sdk
from globus_sdk import AccessTokenAuthorizer, TransferClient
from globus_sdk.scopes import TransferScopes

""" 

See:
	https://globus-sdk-python.readthedocs.io/en/stable/examples/minimal_transfer_script/index.html#example-minimal-transfer

"""

globus_endpoint_gl_homedir='e0370902-9f48-11e9-821b-02b7a92d8e58'
globus_endpoint_kdiba_lab_turbo='8c185a84-5c61-4bbc-b12b-11430e20010f'
endpoint_LNX00052_Fedora:str = 'af3fcfce-f664-11ed-9a7d-83ef71fbf0ae'
endpoint_Apogee:str = '84991054-07b4-11ed-8d83-a54cf61939f8'

# Define the Globus transfer client and authenticate with a user's tokens
auth_client = globus_sdk.NativeAppAuthClient('769d24e1-d1cc-4198-9ff7-2626485da449')

def login_and_get_transfer_client(*, scopes='openid profile email urn:globus:auth:scope:transfer.api.globus.org:all', copy_to_clipboard=True):
	""" 
		'openid profile email urn:globus:auth:scope:transfer.api.globus.org:all'

	"""
	auth_client.oauth2_start_flow(refresh_tokens=True, requested_scopes=scopes)

	authorize_url = auth_client.oauth2_get_authorize_url()
	print(f'Please go to this URL and login: {authorize_url}')
	if copy_to_clipboard:
		df = pd.DataFrame([authorize_url])
		df.to_clipboard(index=False, header=False, sep=',')
		print(f'\t Copied url to clipboard!')

	auth_code = input('Please enter the code you get after login here: ').strip()
	token_response = auth_client.oauth2_exchange_code_for_tokens(auth_code)

	# globus_auth_data = token_response.by_resource_server["auth.globus.org"]
	# get credentials for the Globus Transfer service
	globus_transfer_data = token_response.by_resource_server["transfer.api.globus.org"]
	# the refresh token and access token are often abbreviated as RT and AT
	transfer_rt = globus_transfer_data["refresh_token"]
	transfer_at = globus_transfer_data["access_token"]
	expires_at_s = globus_transfer_data["expires_at_seconds"]

	# construct a RefreshTokenAuthorizer
	# note that `client` is passed to it, to allow it to do the refreshes
	authorizer = globus_sdk.RefreshTokenAuthorizer(transfer_rt, auth_client, access_token=transfer_at, expires_at=expires_at_s)

	# and try using `tc` to make TransferClient calls. Everything should just
	# work -- for days and days, months and months, even years
	transfer_client = globus_sdk.TransferClient(authorizer=authorizer)



	return transfer_client

transfer_client = login_and_get_transfer_client()

# # most specifically, you want these tokens as strings
# AUTH_TOKEN = globus_auth_data["access_token"]
# TRANSFER_TOKEN = globus_transfer_data["access_token"]

# # a GlobusAuthorizer is an auxiliary object we use to wrap the token. In
# # more advanced scenarios, other types of GlobusAuthorizers give us
# # expressive power
# authorizer = globus_sdk.AccessTokenAuthorizer(TRANSFER_TOKEN)
# transfer_client = globus_sdk.TransferClient(authorizer=authorizer)

# 'urn:globus:auth:scope:transfer.api.globus.org:all[*https://auth.globus.org/scopes/8c185a84-5c61-4bbc-b12b-11430e20010f/data_access]'


Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=769d24e1-d1cc-4198-9ff7-2626485da449&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=VOOYT62zgl0JaeA6pzj-vhdK1jdiLRhVAzUcjIVdgRE&code_challenge_method=S256&access_type=offline
	 Copied url to clipboard!


In [3]:
# high level interface; provides iterators for list responses
print("My Endpoints:")
for ep in transfer_client.endpoint_search(filter_scope="my-endpoints"):
    print("[{}] {}".format(ep["id"], ep["display_name"]))


My Endpoints:
[c3a90494-2555-11ec-a47d-a50ad076c282] aleinbook-linux
[84991054-07b4-11ed-8d83-a54cf61939f8] Apogee
[6d0251c4-2585-11ec-9e35-3df4ed83d858] Cubix Win10
[c3578f36-42a8-11ec-a50f-b537d6c07c1d] FlatEdge_Server
[af3fcfce-f664-11ed-9a7d-83ef71fbf0ae] LNX00052_Fedora
[20c84240-1eb1-11eb-81b7-0e2f230cc907] Pho Personal Laptop
[560d3a12-be20-11ed-9916-cb2cff506ca5] Pho_Personal_Testix
[debb635c-2556-11ec-a47d-a50ad076c282] rMBP Pink Dot
[e414f584-2556-11ec-a0a7-6b21ca6daf73] rMBP Pink Dot
[41b8fb12-516b-11ee-8a4b-b1038a0991ac] rMBP-PinkDot


In [ ]:
tasks_list = connect_man.get_tasks()
tasks_list_df = tasks_list.to_dataframe()




## Old Not Working (as of 2023-07-12)

In [ ]:

endpoint_LNX00052_Fedora:str = 'af3fcfce-f664-11ed-9a7d-83ef71fbf0ae'
endpoint_Apogee:str = '84991054-07b4-11ed-8d83-a54cf61939f8'

# Set your Globus access token and endpoint information
access_token = '769d24e1-d1cc-4198-9ff7-2626485da449'
# Create a Globus Transfer client using the access token
authorizer = AccessTokenAuthorizer(access_token)
tc = TransferClient(authorizer=authorizer)


In [ ]:


source_endpoint_id = globus_endpoint_kdiba_lab_turbo
start_time = datetime.datetime(2023, 5, 30, 0, 0)  # Change this to your desired start time

# List the files on the endpoint
# try to submit the task
# if it fails, catch the error...
try:
    response = tc.operation_ls(source_endpoint_id, path='/')
except globus_sdk.TransferAPIError as err:
    # if the error is something other than consent_required, reraise it,
    # exiting the script with an error message
    if not err.info.consent_required:
        raise

    # we now know that the error is a ConsentRequired
    # print an explanatory message and do the login flow again
    print( "Encountered a ConsentRequired error.\n" "You must login a second time to grant consents.\n\n" )
    print(f'required scopes:\n{err.info.consent_required.required_scopes}\n')
    transfer_client = login_and_get_transfer_client(scopes=err.info.consent_required.required_scopes)

In [ ]:
response = tc.operation_ls(source_endpoint_id, path='/')

if response['code'] == "Success":
    # Filter the files based on the modified timestamp
    changed_files = [
        item["name"]
        for item in response["DATA"]
        if item["type"] == "file" and datetime.datetime.fromisoformat(item["last_modified"]) >= start_time
    ]
    print("Changed Files:")
    for file in changed_files:
        print(file)
else:
    print("Failed to list files:", response['message'])


# 2023-07-12 - New Filelist Transfer

In [3]:
from typing import List
from pathlib import Path
import pandas as pd

def convert_filelist_to_new_parent(filelist_source: List[Path], original_parent_path: Path = Path(r'/media/MAX/cloud/turbo/Data'), dest_parent_path: Path = Path(r'/media/MAX/Data')):
    """ Converts a list of file paths from their current parent, specified by `original_parent_path`, to their new parent `dest_parent_path` """
    filelist_dest = []
    for path in filelist_source:
        relative_path = str(path.relative_to(original_parent_path))
        new_path = Path(dest_parent_path) / relative_path
        filelist_dest.append(new_path)
    return filelist_dest

# Load filelist from disk
active_filelist_path = Path('/home/halechr/repo/PhoGlobusHelpers/filelists/session_results_filelist_2023-07-12.csv').resolve()

filelist_df = pd.read_csv(active_filelist_path)
filelist_source = [Path(a_path_str).resolve() for a_path_str in filelist_df.Path]



source_parent_path = Path(r'/media/MAX/cloud/turbo/Data')
dest_parent_path = Path(r'/media/MAX/Data')
# # Build the destination filelist from the source_filelist and the two paths:
filelist_dest = convert_filelist_to_new_parent(filelist_source, original_parent_path=source_parent_path, dest_parent_path=dest_parent_path)
filelist_dest

# filelist_source
# filelist_dest

[PosixPath('/media/MAX/Data/KDIBA/pin01/one/fet11-03_20-28-3/output/global_computation_results.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/pin01/one/fet11-01_12-58-54/output/global_computation_results.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/pin01/one/11-09_22-4-5/output/global_computation_results.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/pin01/one/11-09_21-17-16/output/global_computation_results.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/pin01/one/11-09_12-15-3/output/global_computation_results.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/pin01/one/11-09_11-43-50/output/global_computation_results.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/pin01/one/11-03_12-3-25/output/global_computation_results.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/pin01/one/11-02_19-28-0/output/global_computation_results.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/pin01/one/11-02_17-46-44/output/global_computation_results.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/vvp01/two/2006-4-28_12-38-13/output/global_computation_results.pkl

In [14]:
import os
from typing import List
from globus_sdk import TransferClient, TransferData



globus_endpoint_gl_homedir='e0370902-9f48-11e9-821b-02b7a92d8e58'
globus_endpoint_kdiba_lab_turbo='8c185a84-5c61-4bbc-b12b-11430e20010f'
endpoint_LNX00052_Fedora:str = 'af3fcfce-f664-11ed-9a7d-83ef71fbf0ae'
endpoint_Apogee:str = '84991054-07b4-11ed-8d83-a54cf61939f8'



def batch_transfer_files(transfer_client: TransferClient, source_endpoint:str, destination_endpoint:str, filelist_source:List, filelist_dest:List, max_single_file_wait_time_seconds=3*60*60):
    """ performs a batch transfer for the files specified in the filelists from source to endpoint.
    # Set your source and destination endpoint IDs
    # source_endpoint = "SOURCE_ENDPOINT_ID"
    # destination_endpoint = "DESTINATION_ENDPOINT_ID"
    # # Define the list of source and destination files
    # filelist_source = [
    #     "/path/to/source/file1.txt",
    #     "/path/to/source/file2.txt",
    #     "/path/to/source/file3.txt"
    # ]

    # filelist_dest = [
    #     "/path/to/destination/file1.txt",
    #     "/path/to/destination/file2.txt",
    #     "/path/to/destination/file3.txt"
    # ]
    """
    # Set your transfer options
    transfer_options = {
        "preserve_timestamp": True
    }

    # Turn them all into strings
    filelist_source = [str(path) for path in filelist_source]
    filelist_dest = [str(path) for path in filelist_dest]

    # Verify that the source and destination lists have the same length
    assert len(filelist_source) == len(filelist_dest), "Error: Source and destination file lists must have the same length."

    # Loop through the file lists and initiate the transfers
    for source_path, destination_path in zip(filelist_source, filelist_dest):
        # Create a TransferData object
        transfer_data = TransferData(
            transfer_client,
            source_endpoint,
            destination_endpoint,
            label="Batch File Transfer",
            sync_level="checksum",
            **transfer_options
        )

        # Add the file transfer to the TransferData object
        transfer_data.add_item(source_path, destination_path)

        # Initiate the transfer
        transfer_result = transfer_client.submit_transfer(transfer_data)

        print(f"Transferring {source_path} to {destination_path}...")

        # Wait for the transfer to complete
        transfer_client.task_wait(transfer_result["task_id"], timeout=max_single_file_wait_time_seconds, polling_interval=10)

    print("All transfers completed successfully.")
    


batch_transfer_files(transfer_client, source_endpoint=endpoint_LNX00052_Fedora, destination_endpoint=endpoint_LNX00052_Fedora, filelist_source:List, filelist_dest:List, max_single_file_wait_time_seconds=3*60*60)


Transferring /media/MAX/cloud/turbo/Data/KDIBA/pin01/one/fet11-03_20-28-3/output/global_computation_results.pkl to /media/MAX/Data/KDIBA/pin01/one/fet11-03_20-28-3/output/global_computation_results.pkl...
Transferring /media/MAX/cloud/turbo/Data/KDIBA/pin01/one/fet11-01_12-58-54/output/global_computation_results.pkl to /media/MAX/Data/KDIBA/pin01/one/fet11-01_12-58-54/output/global_computation_results.pkl...
Transferring /media/MAX/cloud/turbo/Data/KDIBA/pin01/one/11-09_22-4-5/output/global_computation_results.pkl to /media/MAX/Data/KDIBA/pin01/one/11-09_22-4-5/output/global_computation_results.pkl...
Transferring /media/MAX/cloud/turbo/Data/KDIBA/pin01/one/11-09_21-17-16/output/global_computation_results.pkl to /media/MAX/Data/KDIBA/pin01/one/11-09_21-17-16/output/global_computation_results.pkl...
Transferring /media/MAX/cloud/turbo/Data/KDIBA/pin01/one/11-09_12-15-3/output/global_computation_results.pkl to /media/MAX/Data/KDIBA/pin01/one/11-09_12-15-3/output/global_computation_result

In [ ]:
'misplaced_loadedSessionPickle_files_2023-07-12.csv'